In [ ]:
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/re_clean_data_low_var_ok.csv')

> **P - Values**

> 越顯著，越有差異

> 越不顯著，越沒有差異

# 自定義

> one_way_anova

> independent_t_test

In [ ]:
def one_way_anova(df, column, target):

  # 使用普通最小二乘法構建線性模型，公式格式為 'target ~ C(column)'，其中 C(column) 表示 column 是類別變量
  model = ols(f'{target} ~ C({column})', data=df).fit()

  # 使用 sm.stats.anova_lm 對模型進行 ANOVA 分析，typ=2 表示使用第二類型的 ANOVA
  anova_table = sm.stats.anova_lm(model, typ=2)

  # 返回 ANOVA 表格
  return anova_table


In [ ]:
def independent_t_test(df, column, target, group1, group2):

  # 從數據框中篩選出 column 等於 group1 的行，並提取 target 列的值
  group1_data = df[df[column] == group1][target]

  # 從數據框中篩選出 column 等於 group2 的行，並提取 target 列的值
  group2_data = df[df[column] == group2][target]

  # 使用 stats.ttest_ind 進行獨立樣本 t 檢驗，返回 t 統計量和 p 值
  t_stat, p_val = stats.ttest_ind(group1_data, group2_data)

  # 返回 t 統計量和 p 值
  return t_stat, p_val

In [ ]:
df.info(max_cols=125)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804044 entries, 0 to 804043
Data columns (total 125 columns):
 #    Column                                Non-Null Count   Dtype   
---   ------                                --------------   -----   
 0    usage_category                        778693 non-null  object  
 1    address                               804044 non-null  object  
 2    property_type                         804044 non-null  object  
 3    community_name                        804044 non-null  int64   
 4    main_building_ratio                   748917 non-null  float64 
 5    city                                  804044 non-null  object  
 6    parking_space_price                   189755 non-null  float64 
 7    transaction_date                      804044 non-null  object  
 8    elevator_available                    804044 non-null  int64   
 9    building_age                          610171 non-null  float64 
 10   number_of_land                        8040

In [ ]:
# 找出類別型變量
categorical_cols = df.select_dtypes(include=['object']).columns

In [ ]:
categorical_cols.size

13

In [ ]:
categorical_cols

Index(['usage_category', 'address', 'property_type', 'city',
       'transaction_date', 'per_ping_calculation_method', 'note',
       'detailed_usage_classification', 'property_b_l_p_type', 'district',
       'cor', 'year_month_for_combine', 'date'],
      dtype='object')

1. usage_category
2. address
3. property_type
4. city
5. transaction_date
6. per_ping_calculation_method
7. note
8. detailed_usage_classification
9. property_b_l_p_type
10. district
11. cor
      

In [ ]:
# 選擇並顯示這些類別型變量的內容
categorical_df = df[categorical_cols]
categorical_df.head(1)

,usage_category,address,property_type,city,transaction_date,per_ping_calculation_method,note,detailed_usage_classification,property_b_l_p_type,district,cor,year_month_for_combine,date
0,住,士林區中社路一段０１１巷００７２號四樓#士林區中社路一段１１巷７２號四樓,公寓(5樓含以下無電梯),A,2016-12-08,總價/總面積,NaN,住家用,房地(土地+建物),A15,"(25.109841388999303, 121.5623636414188)",2016-12-01,2016-12-31


# building_age_group

In [ ]:
df['building_age_group'] = pd.cut(df['building_age'], bins=[0, 10, 20, 30, 40, 50, 60, 70, float('inf')], labels=['0-10年', '10-20年', '20-30年', '30-40年', '40-50年', '50-60年', '60-70年', '大於70年'], right=False)

In [ ]:
# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 將數據框按 'building_age_group' 分組，並提取每組 'new_per_ping' 列的數值
grouped = [group['new_per_ping'].values for name, group in df.groupby('building_age_group')]

# 使用 Levene's Test 檢查各組別之間的方差齊性
levene_stat, levene_p = stats.levene(*grouped)

# 打印 Levene's Test 的統計量和 p 值
print(f"Levene's test statistic: {levene_stat:.6f}, p-value: {levene_p:.6f}")

Levene's test statistic: 635.638912, p-value: 0.000000


In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 創建一個空字典來存儲 Shapiro-Wilk 檢驗的結果
shapiro_results = {}

# 遍歷每個組別，按 'building_age_group' 分組
for name, group in df.groupby('building_age_group'):
    # 對每個組別中的 'new_per_ping' 列進行 Shapiro-Wilk 檢驗
    shapiro_stat, shapiro_p = stats.shapiro(group['new_per_ping'])

    # 將檢驗結果存儲在字典中，以組別名稱為鍵
    shapiro_results[name] = (shapiro_stat, shapiro_p)

    # 打印每個組別的 Shapiro-Wilk 檢驗統計量和 p 值
    print(f"Group {name}: Shapiro-Wilk test statistic: {shapiro_stat:.6f}, p-value: {shapiro_p:.6f}")

Group 0-10年: Shapiro-Wilk test statistic: 0.860813, p-value: 0.000000
Group 10-20年: Shapiro-Wilk test statistic: 0.879425, p-value: 0.000000
Group 20-30年: Shapiro-Wilk test statistic: 0.892739, p-value: 0.000000
Group 30-40年: Shapiro-Wilk test statistic: 0.879796, p-value: 0.000000
Group 40-50年: Shapiro-Wilk test statistic: 0.845397, p-value: 0.000000
Group 50-60年: Shapiro-Wilk test statistic: 0.811487, p-value: 0.000000
Group 60-70年: Shapiro-Wilk test statistic: 0.812847, p-value: 0.000000
Group 大於70年: Shapiro-Wilk test statistic: 0.845273, p-value: 0.000000


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 使用 one_way_anova 函數執行 ANOVA 分析
anova_results = one_way_anova(df, 'building_age_group', 'new_per_ping')

# 打印 ANOVA 結果
print("ANOVA Results for building_age_group:\n", anova_results)

ANOVA Results for building_age_group:
                                         sum_sq            df           F  \
C(building_age_group)   981425763744040.125000      7.000000 2724.922381   
Residual              31394324811584672.000000 610163.000000         NaN   

                        PR(>F)  
C(building_age_group) 0.000000  
Residual                   NaN  


>

# detailed_usage_classification

## 用全部資料分組與分析

In [ ]:
# 創建一個新的欄位來標記分組
df['usage_group'] = df['detailed_usage_classification'].apply(
    lambda x: x if x in ['住家用', '住商用'] else '其他'
)

In [ ]:
# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 將數據框按 '住家用', '住商用' 分組，並提取每組 'new_per_ping' 列的數值
grouped = [group['new_per_ping'].values for name, group in df.groupby('usage_group')]

# 使用 Levene's Test 檢查各組別之間的方差齊性
levene_stat, levene_p = stats.levene(*grouped)

# 打印 Levene's Test 的統計量和 p 值
print(f"Levene's test statistic: {levene_stat:.6f}, p-value: {levene_p:.6f}")

Levene's test statistic: 3809.467302, p-value: 0.000000


In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 創建一個空字典來存儲 Shapiro-Wilk 檢驗的結果
shapiro_results = {}

# 遍歷每個組別，按 'usage_group' 分組
for name, group in df.groupby('usage_group'):
    # 對每個組別中的 'new_per_ping' 列進行 Shapiro-Wilk 檢驗
    shapiro_stat, shapiro_p = stats.shapiro(group['new_per_ping'])

    # 將檢驗結果存儲在字典中，以組別名稱為鍵
    shapiro_results[name] = (shapiro_stat, shapiro_p)

    # 打印每個組別的 Shapiro-Wilk 檢驗統計量和 p 值
    print(f"Group {name}: Shapiro-Wilk test statistic: {shapiro_stat:.6f}, p-value: {shapiro_p:.6f}")

Group 住商用: Shapiro-Wilk test statistic: 0.786369, p-value: 0.000000
Group 住家用: Shapiro-Wilk test statistic: 0.865015, p-value: 0.000000
Group 其他: Shapiro-Wilk test statistic: 0.823280, p-value: 0.000000


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


## 篩選兩組 ( 住家用、住商用 ) 資料出來分析

In [ ]:
usage_df = df[df['detailed_usage_classification'].isin(['住家用', '住商用'])]

In [ ]:
# 檢查 detailed_usage_classification 和 usage_group 是否相同
usage_df.loc[:, 'is_equal'] = usage_df['detailed_usage_classification'] == usage_df['usage_group']

<ipython-input-19-325971d5c8c1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usage_df.loc[:, 'is_equal'] = usage_df['detailed_usage_classification'] == usage_df['usage_group']


In [ ]:
# 打印結果
print(usage_df[['detailed_usage_classification', 'usage_group', 'is_equal']])

       detailed_usage_classification usage_group  is_equal
0                                住家用         住家用      True
1                                住家用         住家用      True
2                                住家用         住家用      True
3                                住家用         住家用      True
4                                住家用         住家用      True
...                              ...         ...       ...
804039                           住家用         住家用      True
804040                           住家用         住家用      True
804041                           住家用         住家用      True
804042                           住家用         住家用      True
804043                           住家用         住家用      True

[703406 rows x 3 columns]


In [ ]:
# 查看是否有 False 值的行
false_rows = usage_df[usage_df['is_equal'] == False]
false_rows

,usage_category,address,property_type,community_name,main_building_ratio,city,parking_space_price,transaction_date,elevator_available,building_age,...,acc_Police_Station_under_750m,acc_Clinic_under_500m,acc_Clinic_under_750m,acc_Pharmacy_under_500m,acc_Pharmacy_under_750m,acc_Medical_Facility_under_500m,acc_Medical_Facility_under_750m,building_age_group,usage_group,is_equal


>　**Levene's Test**

In [ ]:
import pandas as pd
from scipy import stats

# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 過濾出 'detailed_usage_classification' 列值為 '住家用' 和 '住商用' 的資料
usage_df = df[df['detailed_usage_classification'].isin(['住家用', '住商用'])]

# 將數據框按 'detailed_usage_classification' 分組，並提取每組 'new_per_ping' 列的數值
grouped = [group['new_per_ping'].values for name, group in usage_df.groupby('detailed_usage_classification')]

# 使用 Levene's Test 檢查各組別之間的方差齊性
levene_stat, levene_p = stats.levene(*grouped)

# 打印 Levene's Test 的統計量和 p 值
print(f"Levene's test statistic: {levene_stat:.6f}, p-value: {levene_p:.6f}")

Levene's test statistic: 943.055394, p-value: 0.000000


In [ ]:
usage_df

,usage_category,address,property_type,community_name,main_building_ratio,city,parking_space_price,transaction_date,elevator_available,building_age,...,acc_Police_Station_under_500m,acc_Police_Station_under_750m,acc_Clinic_under_500m,acc_Clinic_under_750m,acc_Pharmacy_under_500m,acc_Pharmacy_under_750m,acc_Medical_Facility_under_500m,acc_Medical_Facility_under_750m,building_age_group,usage_group
0,住,士林區中社路一段０１１巷００７２號四樓#士林區中社路一段１１巷７２號四樓,公寓(5樓含以下無電梯),0,100.000000,A,NaN,2016-12-08,0,41.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40-50年,住家用
1,住,士林區後港街０２０巷００１８號四樓#士林區後港街２０巷１８號四樓,公寓(5樓含以下無電梯),0,86.100000,A,NaN,2016-12-03,0,38.000000,...,0.000000,1.000000,21.000000,30.000000,7.000000,10.000000,0.000000,0.000000,30-40年,住家用
2,住,士林區中正路１０４巷００１弄０００２號四樓#士林區中正路１０４巷１弄２號四樓,華廈(10層含以下有電梯),0,49.540000,A,NaN,2016-12-02,0,18.000000,...,0.000000,1.000000,17.000000,40.000000,0.000000,6.000000,0.000000,0.000000,10-20年,住家用
3,住,士林區中山北路七段１９０巷００１６之３號７樓#士林區中山北路七段１９０巷１６之３號７樓,華廈(10層含以下有電梯),1,72.920000,A,NaN,2016-12-10,0,29.000000,...,1.000000,1.000000,6.000000,13.000000,2.000000,2.000000,0.000000,1.000000,20-30年,住家用
4,住,士林區延平北路五段２８５巷００８９號四樓#士林區延平北路五段２８５巷８９號四樓,套房(1房(1廳)1衛),1,51.190000,A,NaN,2016-12-06,0,NaN,...,0.000000,1.000000,34.000000,42.000000,14.000000,18.000000,0.000000,1.000000,NaN,住家用
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804039,住,三重區仁愛街４４５巷００２６號三樓#三重區仁愛街４４５巷２６號三樓,公寓(5樓含以下無電梯),0,74.520000,F,NaN,2022-01-02,0,37.000000,...,1.000000,2.000000,29.000000,39.000000,15.000000,20.000000,0.000000,0.000000,30-40年,住家用
804040,住,三重區頂文路０００３號三樓#三重區頂文路３號三樓,住宅大樓(11層含以上有電梯),1,56.890000,F,NaN,2022-01-18,1,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,住家用
804041,住,三重區富福街００５７號五樓#三重區富福街５７號五樓,公寓(5樓含以下無電梯),0,90.520000,F,NaN,2022-01-27,0,36.000000,...,0.000000,2.000000,20.000000,37.000000,9.000000,18.000000,0.000000,0.000000,30-40年,住家用
804042,住,三重區頂文路００１１號四樓#三重區頂文路１１號四樓,住宅大樓(11層含以上有電梯),1,43.840000,F,225.000000,2022-01-23,1,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,住家用


In [ ]:
grouped

[array([607849.95, 407166.12, 355294.93, ..., 381861.58, 534079.35,
        388609.72]),
 array([384430.56, 299907.15, 189573.46, ..., 319669.05, 448326.98,
        391991.99])]

> **Shapiro-Wilk 檢驗**

In [ ]:
import pandas as pd
from scipy import stats

# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 過濾出 'detailed_usage_classification' 列值為 '住家用' 和 '住商用' 的資料
usage_df = df[df['detailed_usage_classification'].isin(['住家用', '住商用'])]

# 創建一個空字典來存儲 Shapiro-Wilk 檢驗的結果
shapiro_results = {}

# 遍歷每個組別，按 'detailed_usage_classification' 分組
for name, group in usage_df.groupby('detailed_usage_classification'):
    # 對每個組別中的 'new_per_ping' 列進行 Shapiro-Wilk 檢驗
    shapiro_stat, shapiro_p = stats.shapiro(group['new_per_ping'])

    # 將檢驗結果存儲在字典中，以組別名稱為鍵
    shapiro_results[name] = (shapiro_stat, shapiro_p)

    # 打印每個組別的 Shapiro-Wilk 檢驗統計量和 p 值
    print(f"Group {name}: Shapiro-Wilk test statistic: {shapiro_stat:.6f}, p-value: {shapiro_p:.6f}")

Group 住商用: Shapiro-Wilk test statistic: 0.786369, p-value: 0.000000
Group 住家用: Shapiro-Wilk test statistic: 0.865015, p-value: 0.000000


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


> ANOVA

In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 使用 one_way_anova 函數執行 ANOVA 分析
anova_results = one_way_anova(usage_df, 'usage_group', 'new_per_ping')

# 打印 ANOVA 結果
print("ANOVA Results for usage_group:\n", anova_results)

ANOVA Results for usage_group:
                                  sum_sq            df           F   PR(>F)
C(usage_group)    55361252739863.429688      1.000000 1129.415604 0.000000
Residual       34479182419623724.000000 703404.000000         NaN      NaN


# property_type

## 先分四類做

> Levene's Test

In [ ]:
import pandas as pd
from scipy import stats

# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 過濾出 'property_type' 列值
property_type_df = df[df['property_type'].isin(['華廈(10層含以下有電梯)',
                          '住宅大樓(11層含以上有電梯)',
                          '公寓(5樓含以下無電梯)',
                          '透天厝'])]

# 將數據框按 'property_type' 分組，並提取每組 'new_per_ping' 列的數值
grouped = [group['new_per_ping'].values
           for name, group in property_type_df.groupby('property_type')]

# 使用 Levene's Test 檢查各組別之間的方差齊性
levene_stat, levene_p = stats.levene(*grouped)

# 打印 Levene's Test 的統計量和 p 值
print(f"Levene's test statistic: {levene_stat:.6f}")
print(f"p-value: {levene_p:.6f}")

Levene's test statistic: 5293.421446
p-value: 0.000000


In [ ]:
property_type_df

,usage_category,address,property_type,community_name,main_building_ratio,city,parking_space_price,transaction_date,elevator_available,building_age,...,acc_Police_Station_under_500m,acc_Police_Station_under_750m,acc_Clinic_under_500m,acc_Clinic_under_750m,acc_Pharmacy_under_500m,acc_Pharmacy_under_750m,acc_Medical_Facility_under_500m,acc_Medical_Facility_under_750m,building_age_group,usage_group
0,住,士林區中社路一段０１１巷００７２號四樓#士林區中社路一段１１巷７２號四樓,公寓(5樓含以下無電梯),0,100.000000,A,NaN,2016-12-08,0,41.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40-50年,住家用
1,住,士林區後港街０２０巷００１８號四樓#士林區後港街２０巷１８號四樓,公寓(5樓含以下無電梯),0,86.100000,A,NaN,2016-12-03,0,38.000000,...,0.000000,1.000000,21.000000,30.000000,7.000000,10.000000,0.000000,0.000000,30-40年,住家用
2,住,士林區中正路１０４巷００１弄０００２號四樓#士林區中正路１０４巷１弄２號四樓,華廈(10層含以下有電梯),0,49.540000,A,NaN,2016-12-02,0,18.000000,...,0.000000,1.000000,17.000000,40.000000,0.000000,6.000000,0.000000,0.000000,10-20年,住家用
3,住,士林區中山北路七段１９０巷００１６之３號７樓#士林區中山北路七段１９０巷１６之３號７樓,華廈(10層含以下有電梯),1,72.920000,A,NaN,2016-12-10,0,29.000000,...,1.000000,1.000000,6.000000,13.000000,2.000000,2.000000,0.000000,1.000000,20-30年,住家用
5,住,士林區士東路０１２３號四樓#士林區士東路１２３號四樓,公寓(5樓含以下無電梯),0,100.000000,A,NaN,2016-12-12,0,42.000000,...,0.000000,0.000000,40.000000,81.000000,10.000000,23.000000,0.000000,1.000000,40-50年,住家用
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804039,住,三重區仁愛街４４５巷００２６號三樓#三重區仁愛街４４５巷２６號三樓,公寓(5樓含以下無電梯),0,74.520000,F,NaN,2022-01-02,0,37.000000,...,1.000000,2.000000,29.000000,39.000000,15.000000,20.000000,0.000000,0.000000,30-40年,住家用
804040,住,三重區頂文路０００３號三樓#三重區頂文路３號三樓,住宅大樓(11層含以上有電梯),1,56.890000,F,NaN,2022-01-18,1,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,住家用
804041,住,三重區富福街００５７號五樓#三重區富福街５７號五樓,公寓(5樓含以下無電梯),0,90.520000,F,NaN,2022-01-27,0,36.000000,...,0.000000,2.000000,20.000000,37.000000,9.000000,18.000000,0.000000,0.000000,30-40年,住家用
804042,住,三重區頂文路００１１號四樓#三重區頂文路１１號四樓,住宅大樓(11層含以上有電梯),1,43.840000,F,225.000000,2022-01-23,1,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,住家用


In [ ]:
grouped

[array([ 474391.12, 1249967.23, 1424727.6 , ...,  519830.57,  448326.98,
         391991.99]),
 array([384430.56, 299907.15, 604356.99, ..., 384319.75, 428055.7 ,
        319669.05]),
 array([189573.46, 551881.41, 504778.24, ..., 348837.21, 360000.  ,
        447702.06]),
 array([ 983032.59, 1359883.44, 1010752.69, ...,  324473.59,  722195.47,
         530868.39])]

跟以下使用【住家用 & 住商用 分資料】對照一下

```
[array([1249967.23, 1424727.6 , 1509251.39, ...,  519830.57,  448326.98,
         391991.99]),
 array([384430.56, 299907.15, 604356.99, ..., 384319.75, 428055.7 ,
        319669.05]),
 array([189573.46, 551881.41, 504778.24, ..., 321285.14, 360000.  ,
        447702.06]),
 array([ 983032.59, 1359883.44, 1010752.69, ...,  665083.14,  722195.47,
         530868.39])]

```



> Shapiro-Wilk test

In [ ]:
import pandas as pd
from scipy import stats

# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 創建一個空字典來存儲 Shapiro-Wilk 檢驗的結果
shapiro_results = {}

# 遍歷每個組別，按 'property_type' 分組
for name, group in property_type_df.groupby('property_type'):
    # 對每個組別中的 'new_per_ping' 列進行 Shapiro-Wilk 檢驗
    shapiro_stat, shapiro_p = stats.shapiro(group['new_per_ping'])

    # 將檢驗結果存儲在字典中，以組別名稱為鍵
    shapiro_results[name] = (shapiro_stat, shapiro_p)

    # 打印每個組別的 Shapiro-Wilk 檢驗統計量和 p 值

    print(f"Group: {name}" )
    print(f"Shapiro-Wilk test statistic: {shapiro_stat:.6f}")
    print(f"p-value: {shapiro_p:.6f}")
    print('=' * 50)

Group: 住宅大樓(11層含以上有電梯)
Shapiro-Wilk test statistic: 0.869497
p-value: 0.000000
Group: 公寓(5樓含以下無電梯)
Shapiro-Wilk test statistic: 0.895922
p-value: 0.000000
Group: 華廈(10層含以下有電梯)
Shapiro-Wilk test statistic: 0.918070
p-value: 0.000000
Group: 透天厝
Shapiro-Wilk test statistic: 0.709913
p-value: 0.000000


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


> ANOVA

In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 使用 one_way_anova 函數執行 ANOVA 分析
anova_results = one_way_anova(property_type_df, 'property_type', 'new_per_ping')

# 打印 ANOVA 結果
print("ANOVA Results for property_type_group:\n", anova_results)

ANOVA Results for property_type_group:
                                    sum_sq            df           F   PR(>F)
C(property_type)   332045866871221.250000      3.000000 2207.101137 0.000000
Residual         37087888945890144.000000 739567.000000         NaN      NaN


## 再分成以下兩類處裡

> (1) 住宅 (11樓以上)、華廈

> - 華廈(10層含以下有電梯)

> - 住宅大樓(11層含以上有電梯)

> (2) 透天、公寓

> - 公寓(5樓含以下無電梯)

> - 透天厝

> Levene's Test

In [ ]:
import pandas as pd
from scipy import stats

# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 過濾出 'property_type' 列值
property_type_df = df[df['property_type'].isin(['華廈(10層含以下有電梯)',
                           '住宅大樓(11層含以上有電梯)',
                           '公寓(5樓含以下無電梯)',
                           '透天厝'])]

# 創建一個新的欄位來標記分組
property_type_df['group'] = property_type_df['property_type'].apply(
    lambda x: '住宅大樓與華廈' if x in ['住宅大樓(11層含以上有電梯)',
                          '華廈(10層含以下有電梯)'] else '透天與公寓')

# 將數據框按 'group' 分組，並提取每組 'new_per_ping' 列的數值
grouped = [group['new_per_ping'].values for name, group in property_type_df.groupby('group')]

# 使用 Levene's Test 檢查各組別之間的方差齊性
levene_stat, levene_p = stats.levene(*grouped)

# 打印 Levene's Test 的統計量和 p 值
print(f"Levene's test statistic: {levene_stat:.6f}")
print(f"p-value: {levene_p:.6f}")


<ipython-input-100-9aaa0e1d3803>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_type_df['group'] = property_type_df['property_type'].apply(


Levene's test statistic: 111.395905
p-value: 0.000000


In [ ]:
grouped

[array([189573.46, 551881.41, 504778.24, ..., 519830.57, 448326.98,
        391991.99]),
 array([384430.56, 299907.15, 604356.99, ..., 384319.75, 428055.7 ,
        319669.05])]

In [ ]:
property_type_df['group'].value_counts()

group
住宅大樓與華廈    536061
透天與公寓      203510
Name: count, dtype: int64

In [ ]:
property_type_df[['property_type', 'group']].value_counts()

property_type    group  
住宅大樓(11層含以上有電梯)  住宅大樓與華廈    407352
公寓(5樓含以下無電梯)     透天與公寓      178935
華廈(10層含以下有電梯)    住宅大樓與華廈    128709
透天厝              透天與公寓       24575
Name: count, dtype: int64

> Shapiro-Wilk test

In [ ]:
import pandas as pd
from scipy import stats

# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 創建一個空字典來存儲 Shapiro-Wilk 檢驗的結果
shapiro_results = {}

# 遍歷每個組別，按 'property_type' 分組
for name, group in property_type_df.groupby('group'):
    # 對每個組別中的 'new_per_ping' 列進行 Shapiro-Wilk 檢驗
    shapiro_stat, shapiro_p = stats.shapiro(group['new_per_ping'])

    # 將檢驗結果存儲在字典中，以組別名稱為鍵
    shapiro_results[name] = (shapiro_stat, shapiro_p)

    # 打印每個組別的 Shapiro-Wilk 檢驗統計量和 p 值

    print(f"Group: {name}" )
    print(f"Shapiro-Wilk test statistic: {shapiro_stat:.6f}")
    print(f"p-value: {shapiro_p:.6f}")
    print('=' * 50)

Group: 住宅大樓與華廈
Shapiro-Wilk test statistic: 0.883860
p-value: 0.000000
Group: 透天與公寓
Shapiro-Wilk test statistic: 0.779277
p-value: 0.000000


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


> ANOVA

In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 創建一個新的欄位來標記分組
property_type_df['group'] = property_type_df['property_type'].apply(
    lambda x: '住宅大樓與華廈' if x in ['住宅大樓(11層含以上有電梯)',
                          '華廈(10層含以下有電梯)'] else '透天與公寓'
)

# 使用 one_way_anova 函數執行 ANOVA 分析
anova_results = one_way_anova(property_type_df, 'property_type', 'new_per_ping')

# 打印 ANOVA 結果
print("ANOVA Results for property_type_group:\n", anova_results)

<ipython-input-66-58fe3f4603ee>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_type_df['group'] = property_type_df['property_type'].apply(


ANOVA Results for property_type_group:
                                    sum_sq            df           F   PR(>F)
C(property_type)   332045866871221.250000      3.000000 2207.101137 0.000000
Residual         37087888945890144.000000 739567.000000         NaN      NaN


In [ ]:
property_type_df['group'].value_counts()

group
住宅大樓與華廈    536061
透天與公寓      203510
Name: count, dtype: int64

## 使用 住家用 & 住商用 分資料

> 再用這份資料做以上的分析



In [ ]:
usage_df.value_counts('detailed_usage_classification')

detailed_usage_classification
住家用    697455
住商用      5951
Name: count, dtype: int64

In [ ]:
# 過濾出 'detailed_usage_classification' 列值為 '住家用' 和 '住商用' 的資料
usage_df = df[df['detailed_usage_classification'].isin(['住家用', '住商用'])]

In [ ]:
# 過濾出 'property_type' 列值
property_type_usage_df = usage_df[usage_df['property_type'].isin(['華廈(10層含以下有電梯)',
                                 '住宅大樓(11層含以上有電梯)',
                                 '公寓(5樓含以下無電梯)',
                                 '透天厝'])]

### 先處理 property_type 四類

In [ ]:
property_type_usage_df.value_counts('property_type')

property_type
住宅大樓(11層含以上有電梯)    367616
公寓(5樓含以下無電梯)       167662
華廈(10層含以下有電梯)      113872
透天厝                 21121
Name: count, dtype: int64

> Levene's Test

In [ ]:
import pandas as pd
from scipy import stats

# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 將數據框按 'property_type' 分組，並提取每組 'new_per_ping' 列的數值
grouped = [group['new_per_ping'].values
           for name, group in property_type_usage_df.groupby('property_type')]

# 使用 Levene's Test 檢查各組別之間的方差齊性
levene_stat, levene_p = stats.levene(*grouped)

# 打印 Levene's Test 的統計量和 p 值
print(f"Levene's test statistic: {levene_stat:.6f}")
print(f"p-value: {levene_p:.6f}")

Levene's test statistic: 4447.090397
p-value: 0.000000


In [ ]:
grouped

[array([1249967.23, 1424727.6 , 1509251.39, ...,  519830.57,  448326.98,
         391991.99]),
 array([384430.56, 299907.15, 604356.99, ..., 384319.75, 428055.7 ,
        319669.05]),
 array([189573.46, 551881.41, 504778.24, ..., 321285.14, 360000.  ,
        447702.06]),
 array([ 983032.59, 1359883.44, 1010752.69, ...,  665083.14,  722195.47,
         530868.39])]

> Shapiro-Wilk test

In [ ]:
import pandas as pd
from scipy import stats

# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 創建一個空字典來存儲 Shapiro-Wilk 檢驗的結果
shapiro_results = {}

# 遍歷每個組別，按 'property_type' 分組
for name, group in property_type_usage_df.groupby('property_type'):
    # 對每個組別中的 'new_per_ping' 列進行 Shapiro-Wilk 檢驗
    shapiro_stat, shapiro_p = stats.shapiro(group['new_per_ping'])

    # 將檢驗結果存儲在字典中，以組別名稱為鍵
    shapiro_results[name] = (shapiro_stat, shapiro_p)

    # 打印每個組別的 Shapiro-Wilk 檢驗統計量和 p 值

    print(f"Group: {name}" )
    print(f"Shapiro-Wilk test statistic: {shapiro_stat:.6f}")
    print(f"p-value: {shapiro_p:.6f}")
    print('=' * 50)

Group: 住宅大樓(11層含以上有電梯)
Shapiro-Wilk test statistic: 0.870011
p-value: 0.000000
Group: 公寓(5樓含以下無電梯)
Shapiro-Wilk test statistic: 0.905539
p-value: 0.000000
Group: 華廈(10層含以下有電梯)
Shapiro-Wilk test statistic: 0.925220
p-value: 0.000000
Group: 透天厝
Shapiro-Wilk test statistic: 0.713435
p-value: 0.000000


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


> ANOVA

In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 使用 one_way_anova 函數執行 ANOVA 分析
anova_results = one_way_anova(property_type_usage_df, 'property_type', 'new_per_ping')

# 打印 ANOVA 結果
print("ANOVA Results for property_type_group:\n", anova_results)

ANOVA Results for property_type_group:
                                    sum_sq            df           F   PR(>F)
C(property_type)   256602261787623.843750      3.000000 1853.268740 0.000000
Residual         30934896184300532.000000 670267.000000         NaN      NaN


### 再分成以下兩類處裡

> (1) 住宅與華廈

> - 華廈(10層含以下有電梯)

> - 住宅大樓(11層含以上有電梯)

> (2) 透天與公寓

> - 公寓(5樓含以下無電梯)

> - 透天厝

> Levene's Test

In [ ]:
import pandas as pd
from scipy import stats

# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format


# 創建一個新的欄位來標記分組
property_type_usage_df['group'] = property_type_usage_df['property_type'].apply(
    lambda x: '住宅大樓與華廈' if x in ['住宅大樓(11層含以上有電梯)',
                          '華廈(10層含以下有電梯)'] else '透天與公寓')

# 將數據框按 'group' 分組，並提取每組 'new_per_ping' 列的數值
property_type_usage_df_grouped = [group['new_per_ping'].values for name, group in property_type_usage_df.groupby('group')]

# 使用 Levene's Test 檢查各組別之間的方差齊性
levene_stat, levene_p = stats.levene(*property_type_usage_df_grouped)

# 打印 Levene's Test 的統計量和 p 值
print(f"Levene's test statistic: {levene_stat:.6f}")
print(f"p-value: {levene_p:.6f}")

<ipython-input-135-98acd281154b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_type_usage_df['group'] = property_type_usage_df['property_type'].apply(


Levene's test statistic: 308.210490
p-value: 0.000000


In [ ]:
# 打印分組結果計數
property_type_usage_df['group'].value_counts()

group
住宅大樓與華廈    481488
透天與公寓      188783
Name: count, dtype: int64

In [ ]:
property_type_df['group'].value_counts()

group
住宅大樓與華廈    536061
透天與公寓      203510
Name: count, dtype: int64

In [ ]:
property_type_usage_df_grouped

[array([189573.46, 551881.41, 504778.24, ..., 519830.57, 448326.98,
        391991.99]),
 array([384430.56, 299907.15, 604356.99, ..., 384319.75, 428055.7 ,
        319669.05])]


跟上面用全部資料處理的對照
```
[array([189573.46, 551881.41, 504778.24, ..., 519830.57, 448326.98,
        391991.99]),
 array([384430.56, 299907.15, 604356.99, ..., 384319.75, 428055.7 ,
        319669.05])]

```



> Shapiro-Wilk test

In [ ]:
import pandas as pd
from scipy import stats

# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 創建一個空字典來存儲 Shapiro-Wilk 檢驗的結果
shapiro_results = {}

# 遍歷每個組別，按 'property_type' 分組
for name, group in property_type_df.groupby('group'):
    # 對每個組別中的 'new_per_ping' 列進行 Shapiro-Wilk 檢驗
    shapiro_stat, shapiro_p = stats.shapiro(group['new_per_ping'])

    # 將檢驗結果存儲在字典中，以組別名稱為鍵
    shapiro_results[name] = (shapiro_stat, shapiro_p)

    # 打印每個組別的 Shapiro-Wilk 檢驗統計量和 p 值

    print(f"Group: {name}" )
    print(f"Shapiro-Wilk test statistic: {shapiro_stat:.6f}")
    print(f"p-value: {shapiro_p:.6f}")
    print('=' * 50)

Group: 住宅大樓與華廈
Shapiro-Wilk test statistic: 0.883860
p-value: 0.000000
Group: 透天與公寓
Shapiro-Wilk test statistic: 0.779277
p-value: 0.000000


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


> ANOVA

In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 創建一個新的欄位來標記分組
property_type_df['group'] = property_type_df['property_type'].apply(
    lambda x: '住宅大樓與華廈' if x in ['住宅大樓(11層含以上有電梯)',
                          '華廈(10層含以下有電梯)'] else '透天與公寓'
)

# 使用 one_way_anova 函數執行 ANOVA 分析
anova_results = one_way_anova(property_type_df, 'property_type', 'new_per_ping')

# 打印 ANOVA 結果
print("ANOVA Results for property_type_group:\n", anova_results)

<ipython-input-140-58fe3f4603ee>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_type_df['group'] = property_type_df['property_type'].apply(


ANOVA Results for property_type_group:
                                    sum_sq            df           F   PR(>F)
C(property_type)   332045866871221.250000      3.000000 2207.101137 0.000000
Residual         37087888945890144.000000 739567.000000         NaN      NaN


# total_floors ( 建物總樓層 )

> 建物總樓層的切分

> - 1 - 2 樓

> - 3 - 5 樓

> - 6 - 10 樓

> - 11 - 15 樓

> - 16 - 20 樓

> - 21 - 29 樓

> - 30 樓以上

## 使用所有資料 ( df )

In [ ]:
df['total_floors_group'] = pd.cut(df['total_floors'],
                    bins=[2, 5, 10, 15, 20, 29, 30, float('inf')],
                    labels=['1-2 樓', '3-5 樓', '6-10 樓', '11-15 樓', '16-20 樓', '21-29 樓', '30 樓以上'], right=False)

In [ ]:
df['total_floors_group'].value_counts()

total_floors_group
3-5 樓      221506
6-10 樓     219135
11-15 樓    141709
1-2 樓      106965
16-20 樓     89881
30 樓以上      12549
21-29 樓      9070
Name: count, dtype: int64

> Levene's Test

In [ ]:
# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 將數據框按 'total_floors_group' 分組，並提取每組 'new_per_ping' 列的數值
grouped = [group['new_per_ping'].values for name, group in df.groupby('total_floors_group')]

# 使用 Levene's Test 檢查各組別之間的方差齊性
levene_stat, levene_p = stats.levene(*grouped)

# 打印 Levene's Test 的統計量和 p 值
print(f"Levene's test statistic: {levene_stat:.6f}, p-value: {levene_p:.6f}")

Levene's test statistic: 1211.894898, p-value: 0.000000


> Shapiro-Wilk test

In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 創建一個空字典來存儲 Shapiro-Wilk 檢驗的結果
shapiro_results = {}

# 遍歷每個組別，按 'total_floors_group' 分組
for name, group in df.groupby('total_floors_group'):
    # 對每個組別中的 'new_per_ping' 列進行 Shapiro-Wilk 檢驗
    shapiro_stat, shapiro_p = stats.shapiro(group['new_per_ping'])

    # 將檢驗結果存儲在字典中，以組別名稱為鍵
    shapiro_results[name] = (shapiro_stat, shapiro_p)

    # 打印每個組別的 Shapiro-Wilk 檢驗統計量和 p 值
    print(f"Group {name}: Shapiro-Wilk test statistic: {shapiro_stat:.6f}, p-value: {shapiro_p:.6f}")

Group 1-2 樓: Shapiro-Wilk test statistic: 0.786211, p-value: 0.000000
Group 3-5 樓: Shapiro-Wilk test statistic: 0.887983, p-value: 0.000000
Group 6-10 樓: Shapiro-Wilk test statistic: 0.881532, p-value: 0.000000
Group 11-15 樓: Shapiro-Wilk test statistic: 0.873339, p-value: 0.000000
Group 16-20 樓: Shapiro-Wilk test statistic: 0.847799, p-value: 0.000000
Group 21-29 樓: Shapiro-Wilk test statistic: 0.894247, p-value: 0.000000
Group 30 樓以上: Shapiro-Wilk test statistic: 0.734344, p-value: 0.000000


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


> ANOVA

In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 使用 one_way_anova 函數執行 ANOVA 分析
anova_results = one_way_anova(df, 'total_floors_group', 'new_per_ping')

# 打印 ANOVA 結果
print("ANOVA Results for building_age_group:\n", anova_results)

ANOVA Results for building_age_group:
                                         sum_sq            df          F  \
C(total_floors_group)   293218857739087.312500      6.000000 924.750379   
Residual              42319890208818240.000000 800808.000000        NaN   

                        PR(>F)  
C(total_floors_group) 0.000000  
Residual                   NaN  


> 兩兩做 ANOVA

> - Group 6-10 樓
> - Group 11-15 樓
> - Group 16-20 樓

## 使用 住家用 & 住商用 分資料 ( usage_df )

In [ ]:
# 過濾出 'detailed_usage_classification' 列值為 '住家用' 和 '住商用' 的資料
usage_df = df[df['detailed_usage_classification'].isin(['住家用', '住商用'])]

In [ ]:
usage_df.value_counts('detailed_usage_classification')

detailed_usage_classification
住家用    697455
住商用      5951
Name: count, dtype: int64

In [ ]:
usage_df['total_floors_group'] = pd.cut(usage_df['total_floors'],
                    bins=[2, 5, 10, 15, 20, 29, 30, float('inf')],
                    labels=['1-2 樓', '3-5 樓', '6-10 樓', '11-15 樓', '16-20 樓', '21-29 樓', '30 樓以上'], right=False)

<ipython-input-152-391c43418b7d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usage_df['total_floors_group'] = pd.cut(usage_df['total_floors'],


In [ ]:
usage_df.value_counts('total_floors_group')

total_floors_group
3-5 樓      198070
6-10 樓     180593
11-15 樓    123719
1-2 樓       97214
16-20 樓     83281
30 樓以上       9961
21-29 樓      8440
Name: count, dtype: int64

> Levene's Test

In [ ]:
# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 將數據框按 'total_floors_group' 分組，並提取每組 'new_per_ping' 列的數值
grouped = [group['new_per_ping'].values for name, group in usage_df.groupby('total_floors_group')]

# 使用 Levene's Test 檢查各組別之間的方差齊性
levene_stat, levene_p = stats.levene(*grouped)

# 打印 Levene's Test 的統計量和 p 值
print(f"Levene's test statistic: {levene_stat:.6f}, p-value: {levene_p:.6f}")

Levene's test statistic: 866.703511, p-value: 0.000000


> Shapiro-Wilk test

In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 創建一個空字典來存儲 Shapiro-Wilk 檢驗的結果
shapiro_results = {}

# 遍歷每個組別，按 'total_floors_group' 分組
for name, group in usage_df.groupby('total_floors_group'):
    # 對每個組別中的 'new_per_ping' 列進行 Shapiro-Wilk 檢驗
    shapiro_stat, shapiro_p = stats.shapiro(group['new_per_ping'])

    # 將檢驗結果存儲在字典中，以組別名稱為鍵
    shapiro_results[name] = (shapiro_stat, shapiro_p)

    # 打印每個組別的 Shapiro-Wilk 檢驗統計量和 p 值
    print(f"Group {name}: Shapiro-Wilk test statistic: {shapiro_stat:.6f}, p-value: {shapiro_p:.6f}")

Group 1-2 樓: Shapiro-Wilk test statistic: 0.805278, p-value: 0.000000
Group 3-5 樓: Shapiro-Wilk test statistic: 0.907188, p-value: 0.000000
Group 6-10 樓: Shapiro-Wilk test statistic: 0.899594, p-value: 0.000000
Group 11-15 樓: Shapiro-Wilk test statistic: 0.877781, p-value: 0.000000
Group 16-20 樓: Shapiro-Wilk test statistic: 0.841748, p-value: 0.000000
Group 21-29 樓: Shapiro-Wilk test statistic: 0.910263, p-value: 0.000000
Group 30 樓以上: Shapiro-Wilk test statistic: 0.749512, p-value: 0.000000


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


> ANOVA

In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 使用 one_way_anova 函數執行 ANOVA 分析
anova_results = one_way_anova(usage_df, 'total_floors_group', 'new_per_ping')

# 打印 ANOVA 結果
print("ANOVA Results for building_age_group:\n", anova_results)

ANOVA Results for building_age_group:
                                         sum_sq            df          F  \
C(total_floors_group)   230659633642373.156250      6.000000 813.138760   
Residual              33154429884508652.000000 701271.000000        NaN   

                        PR(>F)  
C(total_floors_group) 0.000000  
Residual                   NaN  


> 兩兩做 ANOVA

> - Group 6-10 樓
> - Group 11-15 樓
> - Group 16-20 樓

In [ ]:
# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 創建一個函數來進行 ANOVA 分析並打印結果
def perform_anova(data, groups, value_col):
    for i in range(len(groups)):
        for j in range(i + 1, len(groups)):
            group_i = groups[i]
            group_j = groups[j]
            subset = data[data['total_floors_group'].isin([group_i, group_j])]
            model = ols(f'{value_col} ~ C(total_floors_group)', data=subset).fit()
            anova_results = sm.stats.anova_lm(model, typ=2)
            print(f"ANOVA Results for {group_i} vs {group_j}:\n", anova_results, "\n")

# 定義要進行 ANOVA 的分組
groups_to_compare = ['6-10 樓', '11-15 樓', '16-20 樓']

# 使用函數進行 ANOVA 分析
perform_anova(usage_df, groups_to_compare, 'new_per_ping')

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 1
  warnings.warn('covariance of constraints does not have full '


ANOVA Results for 6-10 樓 vs 11-15 樓:
                                         sum_sq            df        F   PR(>F)
C(total_floors_group)       15834124734.457302      6.000000 0.061397 0.804301
Residual              13080012771031338.000000 304309.000000      NaN      NaN 



/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 1
  warnings.warn('covariance of constraints does not have full '


ANOVA Results for 6-10 樓 vs 16-20 樓:
                                         sum_sq            df        F   PR(>F)
C(total_floors_group)      542456943877.269043      6.000000 2.017907 0.155454
Residual              11822368378736572.000000 263871.000000      NaN      NaN 

ANOVA Results for 11-15 樓 vs 16-20 樓:
                                        sum_sq            df        F   PR(>F)
C(total_floors_group)    1273791331054.007812      6.000000 5.713434 0.016836
Residual              7691549474354881.000000 206997.000000      NaN      NaN 



/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 1
  warnings.warn('covariance of constraints does not have full '


In [ ]:
# 定義要進行 ANOVA 的分組
groups_to_compare_30 = ['16-20 樓', '21-29 樓', '30 樓以上']

# 使用函數進行 ANOVA 分析
perform_anova(usage_df, groups_to_compare_30, 'new_per_ping')

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 1
  warnings.warn('covariance of constraints does not have full '


ANOVA Results for 16-20 樓 vs 21-29 樓:
                                        sum_sq           df        F   PR(>F)
C(total_floors_group)     724237780134.274536     6.000000 3.220824 0.072710
Residual              3437300165328643.000000 91718.000000      NaN      NaN 

ANOVA Results for 16-20 樓 vs 30 樓以上:
                                        sum_sq           df        F   PR(>F)
C(total_floors_group)       3774142704.992291     6.000000 0.013900 0.906147
Residual              4219317730276513.500000 93240.000000      NaN      NaN 

ANOVA Results for 21-29 樓 vs 30 樓以上:
                                        sum_sq           df         F   PR(>F)
C(total_floors_group)    5536022437219.371094     6.000000 13.870594 0.000196
Residual              1223833002897719.750000 18398.000000       NaN      NaN 



/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 1
  warnings.warn('covariance of constraints does not have full '


> 像 ， p-value < 5%

> - 6 - 20

> - 21 - 30

In [ ]:
# 過濾出 'detailed_usage_classification' 列值為 '住家用' 和 '住商用' 的資料
usage_df = df[df['detailed_usage_classification'].isin(['住家用', '住商用'])]

In [ ]:
# 將 'total_floors' 分組
usage_df['total_floors_group'] = pd.cut(usage_df['total_floors'],
                    bins=[1, 5, 30, float('inf')],
                    labels=['1-5 樓', '6-29 樓', '30 樓以上'], right=False)

<ipython-input-188-d3108997866b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usage_df['total_floors_group'] = pd.cut(usage_df['total_floors'],


In [ ]:
# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 創建一個函數來進行 ANOVA 分析並打印結果
def perform_anova(data, groups, value_col):
    for i in range(len(groups)):
        for j in range(i + 1, len(groups)):
            group_i = groups[i]
            group_j = groups[j]
            subset = data[data['total_floors_group'].isin([group_i, group_j])]
            model = ols(f'{value_col} ~ C(total_floors_group)', data=subset).fit()
            anova_results = sm.stats.anova_lm(model, typ=2)
            print(f"ANOVA Results for {group_i} vs {group_j}:\n", anova_results, "\n")

# 定義要進行 ANOVA 的分組
groups_to_compare_30 = ['1-5 樓', '6-29 樓', '30 樓以上']

# 使用函數進行 ANOVA 分析
perform_anova(usage_df, groups_to_compare_30, 'new_per_ping')

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '


ANOVA Results for 1-5 樓 vs 6-29 樓:
                                         sum_sq            df          F  \
C(total_floors_group)    62209246074551.312500      2.000000 651.677833   
Residual              33047721793302320.000000 692388.000000        NaN   

                        PR(>F)  
C(total_floors_group) 0.000000  
Residual                   NaN   



/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '


ANOVA Results for 1-5 樓 vs 30 樓以上:
                                        sum_sq            df          F  \
C(total_floors_group)  153379827480540.937500      2.000000 981.802218   
Residual              8455243070270974.000000 108246.000000        NaN   

                        PR(>F)  
C(total_floors_group) 0.000000  
Residual                   NaN   

ANOVA Results for 6-29 樓 vs 30 樓以上:
                                          sum_sq            df              F  \
C(total_floors_group) 100087032178352256.000000      2.000000 1136549.740579   
Residual               26597460096157808.000000 604061.000000            NaN   

                        PR(>F)  
C(total_floors_group) 0.000000  
Residual                   NaN   



# 購買樓層 v.s. 總樓層

> 這裡用 T - test > 可能不行

In [ ]:
# 過濾掉 NaN 值
df_filtered = df.dropna(subset=['target_floor', 'total_floors'])

# 執行配對樣本 t 檢驗
t_stat, p_value = stats.ttest_rel(df_filtered['target_floor'], df_filtered['total_floors'])

# 打印 t 檢驗的結果
print(f"Paired t-test statistic: {t_stat:.6f}")
print(f"p-value: {p_value:.6f}")

Paired t-test statistic: -920.523740
p-value: 0.000000


# target_floor ( 購買樓層 )

## 使用所有資料 ( df )

In [ ]:
df['target_floor_group'] = pd.cut(df['target_floor'],
                  bins=[2, 5, 10, 15, 20, 29, 30, float('inf')],
                  labels=['1-2 樓', '3-5 樓', '6-10 樓', '11-15 樓',
                    '16-20 樓', '21-29 樓', '30 樓以上'],
                  right=False)

In [ ]:
df['target_floor_group'].value_counts()

target_floor_group
1-2 樓      268446
3-5 樓      242963
6-10 樓     131031
11-15 樓     43606
16-20 樓     22494
30 樓以上       1866
21-29 樓       652
Name: count, dtype: int64

> Levene's Test

In [ ]:
# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 將數據框按 'target_floor_group' 分組，並提取每組 'new_per_ping' 列的數值
grouped = [group['new_per_ping'].values for name, group in df.groupby('target_floor_group')]

# 使用 Levene's Test 檢查各組別之間的方差齊性
levene_stat, levene_p = stats.levene(*grouped)

# 打印 Levene's Test 的統計量和 p 值
print(f"Levene's test statistic: {levene_stat:.6f}, p-value: {levene_p:.6f}")

Levene's test statistic: 202.200758, p-value: 0.000000


> Shapiro-Wilk test

In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 創建一個空字典來存儲 Shapiro-Wilk 檢驗的結果
shapiro_results = {}

# 遍歷每個組別，按 'target_floor_group' 分組
for name, group in df.groupby('target_floor_group'):
    # 對每個組別中的 'new_per_ping' 列進行 Shapiro-Wilk 檢驗
    shapiro_stat, shapiro_p = stats.shapiro(group['new_per_ping'])

    # 將檢驗結果存儲在字典中，以組別名稱為鍵
    shapiro_results[name] = (shapiro_stat, shapiro_p)

    # 打印每個組別的 Shapiro-Wilk 檢驗統計量和 p 值
    print(f"Group {name}: Shapiro-Wilk test statistic: {shapiro_stat:.6f}, p-value: {shapiro_p:.6f}")

Group 1-2 樓: Shapiro-Wilk test statistic: 0.918866, p-value: 0.000000
Group 3-5 樓: Shapiro-Wilk test statistic: 0.907564, p-value: 0.000000
Group 6-10 樓: Shapiro-Wilk test statistic: 0.882685, p-value: 0.000000
Group 11-15 樓: Shapiro-Wilk test statistic: 0.845975, p-value: 0.000000
Group 16-20 樓: Shapiro-Wilk test statistic: 0.786159, p-value: 0.000000
Group 21-29 樓: Shapiro-Wilk test statistic: 0.762048, p-value: 0.000000
Group 30 樓以上: Shapiro-Wilk test statistic: 0.665461, p-value: 0.000000


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 使用 one_way_anova 函數執行 ANOVA 分析
anova_results = one_way_anova(df, 'target_floor_group', 'new_per_ping')

# 打印 ANOVA 結果
print("ANOVA Results for building_age_group:\n", anova_results)

ANOVA Results for building_age_group:
                                         sum_sq            df          F  \
C(target_floor_group)   112488574993454.109375      6.000000 433.527118   
Residual              30749754123805104.000000 711051.000000        NaN   

                        PR(>F)  
C(target_floor_group) 0.000000  
Residual                   NaN  


## 使用 住家用 & 住商用 分資料 ( usage_df )

In [ ]:
# 過濾出 'detailed_usage_classification' 列值為 '住家用' 和 '住商用' 的資料
usage_df = df[df['detailed_usage_classification'].isin(['住家用', '住商用'])]

In [ ]:
# 將 'target_floor_group' 分組
usage_df['target_floor_group'] = pd.cut(usage_df['target_floor'],
                    bins=[1, 5, 30, float('inf')],
                    labels=['1-5 樓', '6-29 樓', '30 樓以上'], right=False)

<ipython-input-184-2a19668f584a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usage_df['target_floor_group'] = pd.cut(usage_df['target_floor'],


> Levene's Test

In [ ]:
# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 將數據框按 'target_floor_group' 分組，並提取每組 'new_per_ping' 列的數值
grouped = [group['new_per_ping'].values for name, group in usage_df.groupby('target_floor_group')]

# 使用 Levene's Test 檢查各組別之間的方差齊性
levene_stat, levene_p = stats.levene(*grouped)

# 打印 Levene's Test 的統計量和 p 值
print(f"Levene's test statistic: {levene_stat:.6f}, p-value: {levene_p:.6f}")

Levene's test statistic: 275.898395, p-value: 0.000000


>　Shapiro-Wilk test

In [ ]:
# 設置顯示浮點數的格式為小數點後六位
pd.options.display.float_format = '{:.6f}'.format

# 創建一個空字典來存儲 Shapiro-Wilk 檢驗的結果
shapiro_results = {}

# 遍歷每個組別，按 'target_floor_group' 分組
for name, group in usage_df.groupby('target_floor_group'):
    # 對每個組別中的 'new_per_ping' 列進行 Shapiro-Wilk 檢驗
    shapiro_stat, shapiro_p = stats.shapiro(group['new_per_ping'])

    # 將檢驗結果存儲在字典中，以組別名稱為鍵
    shapiro_results[name] = (shapiro_stat, shapiro_p)

    # 打印每個組別的 Shapiro-Wilk 檢驗統計量和 p 值
    print(f"Group {name}: Shapiro-Wilk test statistic: {shapiro_stat:.6f}, p-value: {shapiro_p:.6f}")

Group 1-5 樓: Shapiro-Wilk test statistic: 0.833734, p-value: 0.000000
Group 6-29 樓: Shapiro-Wilk test statistic: 0.886789, p-value: 0.000000
Group 30 樓以上: Shapiro-Wilk test statistic: 0.690005, p-value: 0.000000


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


> 分組比對的 ANOVA

In [ ]:
# 創建一個函數來進行 ANOVA 分析並打印結果
def target_perform_anova(data, groups, value_col):
    for i in range(len(groups)):
        for j in range(i + 1, len(groups)):
            group_i = groups[i]
            group_j = groups[j]
            subset = data[data['target_floor_group'].isin([group_i, group_j])]
            model = ols(f'{value_col} ~ C(target_floor_group)', data=subset).fit()
            anova_results = sm.stats.anova_lm(model, typ=2)
            print(f"ANOVA Results for {group_i} vs {group_j}:\n", anova_results, "\n")

# 定義要進行 ANOVA 的分組
groups_to_compare_30 = ['1-5 樓', '6-29 樓', '30 樓以上']

# 使用函數進行 ANOVA 分析
target_perform_anova(usage_df, groups_to_compare_30, 'new_per_ping')

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '


ANOVA Results for 1-5 樓 vs 6-29 樓:
                                         sum_sq            df         F  \
C(target_floor_group)     2765993236010.691406      2.000000 28.319900   
Residual              34159404925961056.000000 699489.000000       NaN   

                        PR(>F)  
C(target_floor_group) 0.000000  
Residual                   NaN   



/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '


ANOVA Results for 1-5 樓 vs 30 樓以上:
                                         sum_sq            df          F  \
C(target_floor_group)    74061838777730.218750      2.000000 669.185190   
Residual              16920272993932058.000000 305766.000000        NaN   

                        PR(>F)  
C(target_floor_group) 0.000000  
Residual                   NaN   

ANOVA Results for 6-29 樓 vs 30 樓以上:
                                         sum_sq            df             F  \
C(target_floor_group) 67060844373743784.000000      2.000000 755698.870288   
Residual              17597574557774548.000000 396609.000000           NaN   

                        PR(>F)  
C(target_floor_group) 0.000000  
Residual                   NaN   



# total_area_ping

## 使用 住家用 & 住商用 分資料 ( usage_df )

In [ ]:
# 過濾出 'detailed_usage_classification' 列值為 '住家用' 和 '住商用' 的資料
usage_df = df[df['detailed_usage_classification'].isin(['住家用', '住商用'])]

In [ ]:
# 將 'total_area_ping' 分組
usage_df['total_area_ping_group'] = pd.cut(usage_df['total_area_ping'],
                    bins=[1, 25, 60, 100, float('inf')],
                    labels=['1-25 坪', '26-60 樓', '61-100 樓','100 樓以上'], right=False)

<ipython-input-206-8a4dfaaf7d09>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usage_df['total_area_ping_group'] = pd.cut(usage_df['total_area_ping'],


In [ ]:
# 將數據框按 'total_area_ping_group' 分組，並提取每組 'new_per_ping' 列的數值
grouped_area = [group['new_per_ping'].values for name, group in usage_df.groupby('total_area_ping_group')]

# 使用 Levene's Test 檢查各組別之間的方差齊性
levene_stat_area, levene_p_area = stats.levene(*grouped_area)

# 打印 Levene's Test 的統計量和 p 值
print(f"Levene's test statistic for total_area_ping_group: {levene_stat_area:.6f}, p-value: {levene_p_area:.6f}")


Levene's test statistic for total_area_ping_group: 5414.600238, p-value: 0.000000


In [ ]:
# 創建一個空字典來存儲 Shapiro-Wilk 檢驗的結果
shapiro_results_area = {}

# 遍歷每個組別，按 'total_area_ping_group' 分組
for name, group in usage_df.groupby('total_area_ping_group'):
    # 對每個組別中的 'new_per_ping' 列進行 Shapiro-Wilk 檢驗
    shapiro_stat_area, shapiro_p_area = stats.shapiro(group['new_per_ping'])

    # 將檢驗結果存儲在字典中，以組別名稱為鍵
    shapiro_results_area[name] = (shapiro_stat_area, shapiro_p_area)

    # 打印每個組別的 Shapiro-Wilk 檢驗統計量和 p 值
    print(f"Group {name}: Shapiro-Wilk test statistic: {shapiro_stat_area:.6f}, p-value: {shapiro_p_area:.6f}")


Group 1-25 坪: Shapiro-Wilk test statistic: 0.882572, p-value: 0.000000
Group 26-60 樓: Shapiro-Wilk test statistic: 0.871510, p-value: 0.000000
Group 61-100 樓: Shapiro-Wilk test statistic: 0.896135, p-value: 0.000000
Group 100 樓以上: Shapiro-Wilk test statistic: 0.831286, p-value: 0.000000


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [ ]:
# 將 'total_area_ping' 分組並使用正確的標籤
usage_df['total_area_ping_group'] = pd.cut(usage_df['total_area_ping'],
                                           bins=[1, 25, 60, 100, float('inf')],
                                           labels=['1-25 坪', '26-60 坪', '61-100 坪', '100 坪以上'],
                                           right=False)

# 檢查分組結果
print(usage_df['total_area_ping_group'].value_counts())



total_area_ping_group
26-60 坪     437108
1-25 坪      174784
61-100 坪     75602
100 坪以上      15912
Name: count, dtype: int64


<ipython-input-212-c1d8f6c658a7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usage_df['total_area_ping_group'] = pd.cut(usage_df['total_area_ping'],


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 設定Pandas顯示浮點數的格式，這裡設置為顯示6位小數
pd.options.display.float_format = '{:.6f}'.format

# 確保分組標籤正確
usage_df['total_area_ping_group'] = pd.cut(usage_df['total_area_ping'],
                                           bins=[1, 25, 60, 100, float('inf')],
                                           labels=['1-25 坪', '26-60 坪', '61-100 坪', '100 坪以上'],
                                           right=False)

# 檢查每個分組的樣本數量
group_counts = usage_df['total_area_ping_group'].value_counts()
print(group_counts)

# 創建函數來進行 ANOVA 分析並打印結果
def area_perform_anova(data, groups, value_col):
    for i in range(len(groups)):
        for j in range(i + 1, len(groups)):
            group_i = groups[i]
            group_j = groups[j]
            subset = data[data['total_area_ping_group'].isin([group_i, group_j])]
            if len(subset) < 2 or subset['total_area_ping_group'].nunique() < 2:
                print(f"Skipping ANOVA for {group_i} vs {group_j} due to insufficient data.")
                continue
            model = ols(f'{value_col} ~ C(total_area_ping_group)', data=subset).fit()
            anova_results = sm.stats.anova_lm(model, typ=2)
            print(f"ANOVA Results for {group_i} vs {group_j}:\n", anova_results, "\n")

# 定義要進行 ANOVA 的分組
groups_to_compare_area = ['1-25 坪', '26-60 坪', '61-100 坪', '100 坪以上']

# 使用函數進行 ANOVA 分析
area_perform_anova(usage_df, groups_to_compare_area, 'new_per_ping')



<ipython-input-214-b2ab1b6e26e6>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usage_df['total_area_ping_group'] = pd.cut(usage_df['total_area_ping'],


total_area_ping_group
26-60 坪     437108
1-25 坪      174784
61-100 坪     75602
100 坪以上      15912
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '


ANOVA Results for 1-25 坪 vs 26-60 坪:
                                            sum_sq            df            F  \
C(total_area_ping_group)  2099956677073320.500000      3.000000 15719.105268   
Residual                 27247998941149536.000000 611890.000000          NaN   

                           PR(>F)  
C(total_area_ping_group) 0.000000  
Residual                      NaN   



/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '


...

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '


ANOVA Results for 26-60 坪 vs 61-100 坪:
                                            sum_sq            df        F  \
C(total_area_ping_group)       11287243218.813585      3.000000 0.095608   
Residual                 20176344896494516.000000 512707.000000      NaN   

                           PR(>F)  
C(total_area_ping_group) 0.757166  
Residual                      NaN   



/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 2
  warnings.warn('covariance of constraints does not have full '


ANOVA Results for 26-60 坪 vs 100 坪以上:
                                             sum_sq            df  \
C(total_area_ping_group) 103423119740776848.000000      3.000000   
Residual                  18931378629703752.000000 453018.000000   

                                     F   PR(>F)  
C(total_area_ping_group) 824953.740826 0.000000  
Residual                           NaN      NaN   

ANOVA Results for 61-100 坪 vs 100 坪以上:
                                           sum_sq           df        F  \
C(total_area_ping_group)       7133146343.778237     3.000000 0.035935   
Residual                 6054994200862590.000000 91511.000000      NaN   

                           PR(>F)  
C(total_area_ping_group) 0.849650  
Residual                      NaN   



/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '


# 分成台北和新北